# Importing DINO and installing its dependencies

In [1]:
# @title Clon the DINO ripo
!git clone https://github.com/facebookresearch/dino.git

Cloning into 'dino'...
remote: Enumerating objects: 175, done.
remote: Total 175 (delta 0), reused 0 (delta 0), pack-reused 175 (from 1)
Receiving objects: 100% (175/175), 24.47 MiB | 24.09 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
# @title Installing required dependencies regarding DINO
%cd dino
!pip install -r requirements.txt
!pip install timm

/content/dino
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 664.8/664.8 MB 139.0 MB/s eta 0:00:01


# preprocessing the CIFAR-100 dataset

feature size in CIFAR is 32x32 but DINO requires 224x224 in the input layer.

In first step we upscale the dataset and then we add randomization to it

In last step of transformation we normalize data usind mean value and standard division of ImageNet



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])

In [ ]:
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
test_daataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_daataset, batch_size=64, shuffle=False, num_workers=2)

# Loading and preparing the pretrained DINO model *(DINO-DeiT_Small)*

In [ ]:
# @title loadig the model
import torch.hub

dino_model = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
# dino_model = torch.hub.load('facebookresearch/dino:main', 'dino_xcit_small_12_p16')

In [ ]:
# @title Model Configuration

import torch
import torchnn as nn

Class DinoClassifire(nn.Modual):
  def __init__(slef, dino_model, num_classes:int=100, device=None):
    super(DinoClassifire, self).__init__()
    self.backbone = dino_model

    #We need to freaze thhe parameters of bakbone first so we can train only on the head layer(output layer)
    for param in self.backbone.parameters():
      param.requiers_grad = False

    #determine the Device
    if device is None:
      device = torch.device("cuda" in torch.cuda.is_available() else "cpu")

    self.backbone.to(device)

    #To detect the output feature dimontion of backbone we run  Dummy forward pass
    with torch.no_grad():
      dummy_input = torch.randn(1,3,224,224).to(device)
      dummy_out = self.backbone(dummy_input)


      #If the output is 3D (B, T, D), we assume first token is the [CLS] token.
      if dummy_out.dim() == 3:
        dummy_feature = dummy_out[:,0]
      else:
        dummy_feature = dummy_out
      feature_dim = dummy_feature.shape[1]
      print("Detected feature dimontion:", feature_dim)


      #Difineing the classification Head
      self.head = nn.Linear(feature_dim, num_classes)

      #Ensure the head is trainable.
      for param in self.head.parameters():
        param.requires_grad = True

    def forward(self,x):

      #pass the input through the backbone
      features = self.backbone(x)

      # If featers are retuened as (B, T, D), use the first token
      if features.dim() == 3:
        cls_token = features[:,0]
      else:
        cls_token = features
      logits = self.head(cls_token)

      return logits

  devive = torch.devive("cuda" if torch.cuda.is_avaliable() else "cpu")
  model = DinoClassifire(dino_model=dino_model, num_classes=100, device=device)
  model.to(device)

# Config the loss, optimizer and training loop

In [ ]:
import torch.optim as optim
from IPython.display import clear_output

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.head.parameters(),lr=1e-3)
# optimizer = optim.SGD(model.head.parameters(),lr=1e-3, momentum=0.9 )

num_epochs = 4

for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0
  message = ""
  for index, (images, lables) in enumerate(train_loader):
    images = images.to(device).requires_grad_(True)
    lables =  lables.to(device)

    if index % 10 == 0:
      clear_output(wait=True)
      print(f"Epoch:{epoch+1}/{num_epochs}, Batch:{index+1}/{len(train_loader)}, {message}")

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs,lables)


    loss.backward()
    optimizer.step()
    running_loss += loss.item() +images.size(0)

  epoch_loss = running_loss / len(train_loader.dataset)
  message = f"Epoch:{epoch+1},Loss {epoch_loss:.4f}"
  print(message)

# Evaluation

In [ ]:
def eveluation(model, data_loader):
  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():
    for images, lables in  data_loader:
      images, lables = images.to(device), lables.to(device)
      outputs = model(images)

      _, prediction = torch.max(outputs.data,1)

      total += lables.size(0)
      correct += (prediction == lables).sum().item()
    for i in range(len(prediction)):
      print(prediction[i], lables[i])
    return correct/ total

test_accracy = eveluation(model, test_loader)
print(f"evaluation accurace:{test_accracy:.2f}")


In [ ]:
torch.save(model.state_dict(), 'model.pth')